
# 🧩 Day 2 — Practical OOP in AI Systems

**Goal:** Build a modular, extensible *text-retrieval* mini‑pipeline that runs fully offline and showcases real OOP techniques you can reuse in agent or RAG systems.

**What you'll practice**
- Abstractions with `abc.ABC` and dependency injection
- **Strategy** for interchangeable components (preprocessors, embedders, retrievers)
- **Adapter** to wrap third‑party or legacy components behind your interface
- **Observer** for lifecycle events + **Mixin** for logging
- **Decorator** for caching & timing
- A tiny but real **TF‑IDF** embedder (implemented from scratch) + **cosine similarity**
- Clean tests and exercises to extend the design



> This lab mirrors the architecture ideas from the previous session (abstractions, composition over inheritance, and patterns) and turns them into a fully working, testable mini‑project.


## 1) Tiny corpus for experimentation

In [1]:

CORPUS = {
    "doc1": "Object-oriented programming (OOP) organizes software as collaborating objects with state and behavior. Composition favors reuse over inheritance.",
    "doc2": "Retrieval-Augmented Generation (RAG) combines a retriever that finds relevant context with a generator that produces answers.",
    "doc3": "TF-IDF represents a document by weighting terms by their frequency and their rarity across the corpus. Cosine similarity compares vectors by angle.",
    "doc4": "Mixins are small classes that provide reusable behaviors via multiple inheritance, such as logging or timing.",
    "doc5": "In AI engineering, modular design enables swapping tokenizers, embedders, vector stores, and rerankers without changing high-level code.",
    "doc6": "Observer pattern broadcasts events to interested listeners, enabling decoupled logging, metrics, and UI updates.",
}
len(CORPUS)


6

## 2) Core interfaces (ABCs)

In [2]:

# from __future__ import annotations
from abc import ABC, abstractmethod
from typing import Dict, List, Iterable, Tuple, Callable, Any
import math
import time
from functools import lru_cache

# ---------- Observer ----------
class EventBus:
    def __init__(self):
        self._subs: Dict[str, List[Callable[[Dict[str, Any]], None]]] = {}
    def subscribe(self, event: str, fn: Callable[[Dict[str, Any]], None]):
        self._subs.setdefault(event, []).append(fn)
    def publish(self, event: str, payload: Dict[str, Any]):
        for fn in self._subs.get(event, []):
            fn(payload)

# ---------- Mixins ----------
class LogMixin:
    def log(self, *args):  # lightweight
        print("[LOG]", *args)

class TimeMixin:
    def timeit(self, label: str, fn: Callable, *a, **k):
        t0 = time.perf_counter()
        out = fn(*a, **k)
        dt = (time.perf_counter() - t0) * 1000
        print(f"[TIMER] {label}: {dt:.2f} ms")
        return out

# ---------- ABCs ----------
class Preprocessor(ABC):
    @abstractmethod
    def clean(self, text: str) -> str: ...

class Embedder(ABC):
    @abstractmethod
    def fit(self, corpus: Iterable[str]) -> None: ...
    @abstractmethod
    def encode(self, text: str) -> List[float]: ...

class VectorIndex(ABC):
    @abstractmethod
    def add(self, doc_id: str, vector: List[float]): ...
    @abstractmethod
    def search(self, query_vec: List[float], k: int = 3) -> List[Tuple[str, float]]: ...

class Retriever(ABC):
    @abstractmethod
    def retrieve(self, query: str, k: int = 3) -> List[Tuple[str, float]]: ...

class Reranker(ABC):
    @abstractmethod
    def rerank(self, query: str, items: List[Tuple[str, float]]) -> List[Tuple[str, float]]: ...

class Answerer(ABC):
    @abstractmethod
    def answer(self, query: str, context_docs: List[str]) -> str: ...


## 3) Concrete strategies and utilities

In [3]:

import re
from collections import Counter

# ---- Utilities ----
def tokenize(text: str) -> List[str]:
    return re.findall(r"[A-Za-z0-9]+", text.lower())

def cosine(a: List[float], b: List[float]) -> float:
    num = sum(x*y for x, y in zip(a, b))
    da = math.sqrt(sum(x*x for x in a))
    db = math.sqrt(sum(y*y for y in b))
    return 0.0 if da == 0 or db == 0 else num / (da * db)

# ---- Preprocessors ----
class BasicPreprocessor(Preprocessor):
    STOPWORDS = {"the","a","an","and","or","to","of","in","is","are","that","with","by"}
    def clean(self, text: str) -> str:
        toks = [t for t in tokenize(text) if t not in self.STOPWORDS]
        return " ".join(toks)

class NoopPreprocessor(Preprocessor):
    def clean(self, text: str) -> str:
        return text

# ---- TF-IDF Embedder (from scratch) ----
class TfidfEmbedder(Embedder, LogMixin):
    def __init__(self):
        self.vocab: Dict[str,int] = {}
        self.idf: List[float] = []
    def fit(self, corpus: Iterable[str]) -> None:
        docs = [tokenize(doc) for doc in corpus]
        vocab = {}
        for doc in docs:
            for t in doc:
                if t not in vocab:
                    vocab[t] = len(vocab)
        self.vocab = vocab
        N = len(docs)
        df = [0]*len(vocab)
        for doc in docs:
            seen = set(doc)
            for t in seen:
                df[vocab[t]] += 1
        import math
        self.idf = [math.log((N+1)/(df_i+1))+1 for df_i in df]
        self.log(f"Fitted TF-IDF on {N} docs, |vocab|={len(vocab)}")
    def encode(self, text: str) -> List[float]:
        toks = tokenize(text)
        tf = Counter(toks)
        vec = [0.0]*len(self.vocab)
        for t, c in tf.items():
            if t in self.vocab:
                idx = self.vocab[t]
                vec[idx] = (c / len(toks)) * self.idf[idx]
        return vec

# ---- Adapter example ----
class LegacyCountVectorizer:
    # Pretend this is a third‑party component with a different API.
    def fit_corpus(self, docs: List[str]):
        self.vocab = {}
        for d in docs:
            for t in tokenize(d):
                self.vocab.setdefault(t, len(self.vocab))
    def vectorize(self, text: str) -> List[float]:
        v = [0.0]*len(self.vocab)
        for t in tokenize(text):
            if t in self.vocab:
                v[self.vocab[t]] += 1.0
        return v

class CountVectorAdapter(Embedder):
    def __init__(self, legacy: LegacyCountVectorizer):
        self.legacy = legacy
    def fit(self, corpus: Iterable[str]) -> None:
        self.legacy.fit_corpus(list(corpus))
    def encode(self, text: str) -> List[float]:
        return self.legacy.vectorize(text)

# ---- Vector index ----
class BruteForceIndex(VectorIndex):
    def __init__(self):
        self._vecs: Dict[str, List[float]] = {}
    def add(self, doc_id: str, vector: List[float]):
        self._vecs[doc_id] = vector
    def search(self, query_vec: List[float], k: int = 3) -> List[Tuple[str, float]]:
        scores = [(doc_id, cosine(query_vec, vec)) for doc_id, vec in self._vecs.items()]
        return sorted(scores, key=lambda x: x[1], reverse=True)[:k]

# ---- Retriever ----
class SimpleRetriever(Retriever):
    def __init__(self, pre: Preprocessor, emb: Embedder, index: VectorIndex, bus: EventBus | None = None):
        self.pre, self.emb, self.index, self.bus = pre, emb, index, bus
    @lru_cache(maxsize=256)
    def _encode_cached(self, text: str) -> List[float]:
        return self.emb.encode(text)
    def retrieve(self, query: str, k: int = 3) -> List[Tuple[str, float]]:
        clean = self.pre.clean(query)
        if self.bus: self.bus.publish("query.start", {"query": query, "clean": clean})
        qv = self._encode_cached(clean)
        hits = self.index.search(qv, k=k)
        if self.bus: self.bus.publish("query.done", {"query": query, "hits": hits})
        return hits

# ---- Reranker (optional; here just identity passthrough) ----
class IdentityReranker(Reranker):
    def rerank(self, query: str, items: List[Tuple[str, float]]) -> List[Tuple[str, float]]:
        return items

# ---- Answerer ----
class TemplateAnswerer(Answerer):
    def __init__(self, corpus: Dict[str,str]):
        self.corpus = corpus
    def answer(self, query: str, context_docs: List[str]) -> str:
        bullets = "\n".join(f"- {doc[:160]}..." for doc in context_docs)
        return f"Answering: '{query}'\n\nTop context:\n{bullets}"


## 4) Ingest & build the index

In [4]:

BUS = EventBus()
BUS.subscribe("query.start", lambda e: print(f"[EVENT] query.start clean='{e['clean']}'"))
BUS.subscribe("query.done",  lambda e: print(f"[EVENT] query.done hits={e['hits']}"))

pre = BasicPreprocessor()
emb = TfidfEmbedder()
emb.fit(CORPUS.values())

index = BruteForceIndex()
for doc_id, text in CORPUS.items():
    index.add(doc_id, emb.encode(pre.clean(text)))

retriever = SimpleRetriever(pre, emb, index, bus=BUS)
reranker  = IdentityReranker()
answerer  = TemplateAnswerer(CORPUS)

print("Index ready. Documents:", list(CORPUS.keys()))


[LOG] Fitted TF-IDF on 6 docs, |vocab|=92
Index ready. Documents: ['doc1', 'doc2', 'doc3', 'doc4', 'doc5', 'doc6']


## 5) End‑to‑end pipeline

In [5]:

def run_pipeline(query: str, k: int = 3):
    hits = retriever.retrieve(query, k=k)
    doc_ids = [doc_id for doc_id, _ in reranker.rerank(query, hits)]
    contexts = [CORPUS[i] for i in doc_ids]
    return answerer.answer(query, contexts)

print(run_pipeline("How does TF-IDF work and why cosine similarity?"))


[EVENT] query.start clean='how does tf idf work why cosine similarity'
[EVENT] query.done hits=[('doc3', 0.4588314677411235), ('doc1', 0.0), ('doc2', 0.0)]
Answering: 'How does TF-IDF work and why cosine similarity?'

Top context:
- TF-IDF represents a document by weighting terms by their frequency and their rarity across the corpus. Cosine similarity compares vectors by angle....
- Object-oriented programming (OOP) organizes software as collaborating objects with state and behavior. Composition favors reuse over inheritance....
- Retrieval-Augmented Generation (RAG) combines a retriever that finds relevant context with a generator that produces answers....


## 6) Simple factory for embedders

In [6]:

def make_embedder(kind: str) -> Embedder:
    kind = kind.lower()
    if kind == "tfidf":
        return TfidfEmbedder()
    elif kind == "count":
        return CountVectorAdapter(LegacyCountVectorizer())
    raise ValueError(f"Unknown embedder kind: {kind}")

# Demo swap:
emb2 = make_embedder("count")
emb2.fit(CORPUS.values())
index2 = BruteForceIndex()
for i, t in CORPUS.items():
    index2.add(i, emb2.encode(pre.clean(t)))
retriever2 = SimpleRetriever(pre, emb2, index2)
print([x for x in retriever2.retrieve("mixins for logging and timing", k=3)])


[('doc4', 0.4803844614152615), ('doc6', 0.16666666666666669), ('doc1', 0.0)]


## 7) Tests

In [7]:

import unittest

class TestMiniPipeline(unittest.TestCase):
    def test_tokenize(self):
        self.assertEqual(tokenize("A,B!"), ["a","b"])
    def test_cosine_unit(self):
        self.assertAlmostEqual(cosine([1,0],[1,0]), 1.0, places=6)
        self.assertAlmostEqual(cosine([1,0],[0,1]), 0.0, places=6)
    def test_retrieval_nonempty(self):
        hits = retriever.retrieve("What is OOP?", k=2)
        self.assertTrue(len(hits) == 2 and all(isinstance(h[1], float) for h in hits))

unittest.TextTestRunner().run(unittest.defaultTestLoader.loadTestsFromTestCase(TestMiniPipeline));


...
----------------------------------------------------------------------
Ran 3 tests in 0.002s

OK


[EVENT] query.start clean='what oop'
[EVENT] query.done hits=[('doc1', 0.2552811336748491), ('doc2', 0.0)]



## 8) Exercises (✍️ you edit these cells)

1. **Add a new Preprocessor strategy** that stems words (you can implement a tiny stemmer that drops common suffixes like `ing`, `ed`, `s`).
2. **Implement a BM25Retriever** (hint: compute IDF as above; use Okapi BM25 formula with `k1=1.2`, `b=0.75`).
3. **Add a `CacheMixin`** that decorates `VectorIndex.search` with an LRU cache keyed by `(hash(query_vec), k)`.
4. **Observer drill:** emit `ingest.start`/`ingest.done` events during indexing and print basic metrics.
5. **Reranker:** implement a query‑document *bigram overlap* reranker and compare with identity.
6. **Adapter:** pretend you received embeddings from a remote vector service shaped as dicts `{"embedding": [...], "dim": N}`. Write `RemoteVectorAdapter` that conforms to `Embedder`.
7. **Testing:** write tests for your new components in section 7.


In [ ]:

# 8.1) New Preprocessor (Stemmer) — TODO
class StemPreprocessor(Preprocessor):
    def clean(self, text: str) -> str:
        # TODO: implement very tiny rule-based stemmer
        return text  # replace


In [ ]:

# 8.2) BM25 Retriever — TODO
class BM25Retriever(Retriever):
    def __init__(self, pre: Preprocessor, index_docs: Dict[str, str]):
        # TODO: precompute bm25 stats
        self.pre = pre
        self.docs = index_docs
    def retrieve(self, query: str, k: int = 3) -> List[Tuple[str, float]]:
        # TODO: compute BM25 scores
        return []


In [ ]:

# 8.3) CacheMixin for search — TODO
class CacheMixin:
    # TODO: wrap an existing index.search with lru_cache
    pass


In [ ]:

# 8.4) Observer instrumentation for ingest — TODO
def build_index_with_events(pre: Preprocessor, emb: Embedder, corpus: Dict[str,str], bus: EventBus) -> VectorIndex:
    # TODO: publish ingest.start / ingest.done with timings and counts
    index = BruteForceIndex()
    return index


In [ ]:

# 8.5) Bigram overlap Reranker — TODO
class BigramOverlapReranker(Reranker):
    def rerank(self, query: str, items: List[Tuple[str, float]]) -> List[Tuple[str, float]]:
        return items


In [ ]:

# 8.6) RemoteVectorAdapter — TODO
class RemoteVectorAdapter(Embedder):
    def fit(self, corpus: Iterable[str]) -> None:
        pass  # pretend remote doesn't need fit
    def encode(self, text: str) -> List[float]:
        # TODO: call a fake remote and adapt to list[float]
        return []



## 9) Bonus — Swappable UIs

Because the pipeline is cleanly separated, you can build multiple clients:
- a CLI (input loop)
- a simple Streamlit/Gradio app
- a batch evaluator that runs a small set of queries and prints top‑k docs

Try it as a challenge after class. 👇


In [ ]:

def cli_demo():
    print("Type a query (or 'quit'):")
    while True:
        q = input("> ").strip()
        if q.lower() in {"q","quit","exit"}:
            break
        print(run_pipeline(q, k=3), "\n")

# Uncomment to try in an interactive session:
# cli_demo()
